# Definition Phrasing

## 0. Requirements

In [330]:
#!pip3 install requests

In [87]:
import pandas as pd
from ast import literal_eval
from collections import Counter
import numpy as np
import requests
import definition_strings as ds

## 1. Create/ Load Definitiontexts

Todo:
- phrase definitions for each main concept
- 

In [88]:
compounds = pd.read_csv("../output/compounds_full.csv")
#compounds = pd.read_csv("../output/test_compounds.csv")

In [89]:
compounds.columns

Index(['original', 'second_part', 'noun_forms', 'lemma', 'genus',
       'compound_forms', 'related_words', 'hypernyms', 'roots', 'en_hypernyms',
       'concept', 'definition', 'path', 'wup', 'stem_cistem', 'stem_porter',
       'stem_lancaster', 'stem_snowball', 'share_cistem', 'share_porter',
       'share_lancaster', 'share_snowball', 'dist_stemmer', 'similar_words',
       'PERS_pro', 'ORG_pro', 'PERS_con', 'ORG_con', 'manual_sentiment',
       'pro_mods', 'con_mods', 'pro_sarcasm', 'con_sarcasm', 'pro_attr',
       'con_attr', 'tf_pro', 'tf_con', 'tfidf_pro', 'tfidf_con'],
      dtype='object')

In [90]:
compounds = compounds[["original", "lemma", "genus", "compound_forms", 
                      "related_words", "hypernyms", "en_hypernyms", "concept",
                      "definition", "similar_words", "PERS_pro", "PERS_con", 
                       "ORG_pro", "ORG_con", "manual_sentiment", "tf_pro", "tf_con",
                       "tfidf_pro", "tfidf_con", "pro_mods", "con_mods", "pro_sarcasm", "con_sarcasm"]]

In [91]:
compounds['compound_forms'] = compounds.compound_forms.apply(lambda x: literal_eval(str(x)))
compounds['related_words'] = compounds.related_words.apply(lambda x: literal_eval(str(x)))
compounds['hypernyms'] = compounds.hypernyms.apply(lambda x: literal_eval(str(x)))
compounds['en_hypernyms'] = compounds.en_hypernyms.apply(lambda x: literal_eval(str(x)) if(str(x) != 'nan') else x)
compounds['definition'] = compounds.definition.apply(lambda x: literal_eval(str(x)))
compounds['PERS_pro'] = compounds.PERS_pro.apply(lambda x: literal_eval(str(x)) if(str(x) != 'nan') else x)
compounds['PERS_con'] = compounds.PERS_con.apply(lambda x: literal_eval(str(x)) if(str(x) != 'nan') else x)
compounds['ORG_pro'] = compounds.ORG_pro.apply(lambda x: literal_eval(str(x)) if(str(x) != 'nan') else x)
compounds['ORG_con'] = compounds.ORG_con.apply(lambda x: literal_eval(str(x)) if(str(x) != 'nan') else x)
compounds['similar_words'] = compounds.similar_words.apply(lambda x: literal_eval(str(x)) if(str(x) != 'nan') else x)
compounds['pro_mods'] = compounds.pro_mods.apply(lambda x: literal_eval(str(x)) if(str(x) != 'nan') else x)
compounds['con_mods'] = compounds.con_mods.apply(lambda x: literal_eval(str(x)) if(str(x) != 'nan') else x)

In [92]:
compounds.PERS_pro = compounds.PERS_pro.apply(lambda x: Counter(x) if(str(x) != 'nan') else x)
compounds.PERS_con = compounds.PERS_con.apply(lambda x: Counter(x) if(str(x) != 'nan') else x)
compounds.ORG_pro = compounds.ORG_pro.apply(lambda x: Counter(x) if(str(x) != 'nan') else x)
compounds.ORG_con = compounds.ORG_con.apply(lambda x: Counter(x) if(str(x) != 'nan') else x)

In [93]:
# replace sentiment with the German counterparts
compounds.manual_sentiment = compounds.manual_sentiment.replace("negative", "negativ")
compounds.manual_sentiment = compounds.manual_sentiment.replace("positive", "positiv")

In [94]:
compounds.pro_mods = compounds.pro_mods.apply(lambda x: Counter(x) if(str(x) != 'nan') else x)
compounds.con_mods = compounds.con_mods.apply(lambda x: Counter(x) if(str(x) != 'nan') else x)

In [95]:
compounds.con_mods[1]

Counter()

In [96]:
print(compounds.con_mods[1].most_common(2))

" und ".join([el[0] for el in compounds['con_mods'][1].most_common(2) if el[1] > 1])

#con_pers = compounds['PERS_con'].iloc[idx].most_common(1)[0]



[]


''

In [340]:
import urllib.request, json

In [349]:
import urllib.request, json 
with urllib.request.urlopen("https://www.dwds.de/api/wb/snippet/?q=Haus") as url:
    data = json.load(url)
    print(data)

#url = "https://www.dwds.de/api/wb/snippet/?q=Haus"
#response = urllib.urlopen(url)
#data = json.loads(response.read())

[{'lemma': 'Haus', 'url': 'https://www.dwds.de/wb/Haus', 'wortart': 'Substantiv', 'input': 'Haus'}]


In [353]:
data[0]["lemma"]

'Haus'

## 2. Create Placeholders

In [166]:
persons_text = """Der Begriff {COMPOUND} bezeichnet eine/n {DEFINITION} im Bezug auf den Klimawandel. 
Der Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet
 und ist {SENTIMENT} konnotiert. Personen, die im Zusammenhang mit dem Begriff erwähnt werden sind 
 {CON_PERS1} und {CON_PERS2} (Con Corpus) und {PRO_PERS1} und {PRO_PERS2} (Pro Corpus). 
 Außerdem werden im Kontext von {COMPOUND} folgende Organisationen genannt: {CON_ORG1} und {CON_ORG2} (C2022), sowie {PRO_ORG1} und {PRO_ORG2} (P2022).

Siehe auch: {SIMILAR_WORDS}"""

## 3. Fill Placeholders

In [ ]:
# add collocation info and sarcasm / attribution info

In [85]:
### INITIATE STRING BASES FOR EACH CATEGORY

str_base_pers =  """Der Begriff {COMPOUND} bezeichnet eine/n {DEFINITION} im Bezug auf den Klimawandel.
\nDer Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet.
\nAuf den gesamten Korpus gesehen, entspricht das einer relativen Häufigkeit (TF-IDF) von {CON_TFIDF} für die Klimaskeptiker und {PRO_TFIDF} für die Klimaaktivisten.
Hierbei verwenden die Klimaskeptiker "{COMPOUND}" im Sinne einer {ATTRIBUTION}.

\nIn unserem Korpus Sample ist der Begriff meist {SENTIMENT} konnotiert.  


"""

str_base_loc = """Der Begriff {COMPOUND} bezeichnet eine/n {DEFINITION} im Bezug auf den Klimawandel.
\nDer Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet und ist {SENTIMENT} konnotiert. """

str_base_action = """{COMPOUND} bezeichnet eine/n {DEFINITION} [Aktion] im Bezug auf den Klimawandel.
\nDer Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet und ist {SENTIMENT} konnotiert. """

str_base_abstract = """Ein {COMPOUND} ist {DEFINITION} im Bezug auf den Klimawandel.
\nDer Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet und ist {SENTIMENT} konnotiert. """

str_base_group = """Als {COMPOUND} wird eine Personengruppe bezeichnet, die {DEFINITION} im Bezug auf den Klimawandel.
\nDer Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet und ist {SENTIMENT} konnotiert. """


# INITIATE STRINGS THAT ARE EQUAL FOR ALL CATEGORIES
str_mods_pro = """\nIm Subdiskurs der Klimaaktivisten wird der Begriff von Wörtern wie {PRO_MODS} modifiziert."""
str_mods_con = """\nAdjektive/Adverbien wie {CON_MODS} treten häufig auf um den Begriff im Subdiskurs der Klimaskeptiker näher zu beschreiben."""
str_pers_con = """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (C2022) """
str_pers_pro = """und {PRO_PERS} (P2022). """
str_org_con = """\nAußerdem werden im Kontext von "{COMPOUND}" folgende Organisationen genannt: {CON_ORG} (C2022)"""
str_org_pro = """und {PRO_ORG} (P2022). """    
str_simwords = """\nSiehe auch: {SIMILAR_WORDS}"""

In [86]:
#def fill_persons(df):
    
for word in compounds.original:
    idx = compounds.index[compounds['original'] == word][0]
    
    if compounds['concept'].iloc[idx] == "person":
        text = str_base_pers        
        #text = """Der Begriff {COMPOUND} bezeichnet eine/n {DEFINITION} im Bezug auf den Klimawandel.\nDer Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet und ist {SENTIMENT} konnotiert. """
        
    elif compounds['concept'].iloc[idx] == "location":
        text = str_base_loc
    
    elif compounds['concept'].iloc[idx] == "group":
        text = str_base_group
        
    elif compounds['concept'].iloc[idx] == "abstraction":
        text = str_base_abstract
        
    elif compounds['concept'].iloc[idx] == "action":
        text = str_base_action
    
        
    # retrieve according information
    compound = word.capitalize()
    con_freq = compounds['tf_con'].iloc[idx]
    pro_freq = compounds['tf_pro'].iloc[idx]
    sentiment = compounds['manual_sentiment'].iloc[idx]
        
    try:
        definition = compounds['definition'].iloc[idx][0] # try [0] for definition
    except:
        print("NO DEFINITION AVAILABLE")
        definition = " "
            
            
    # if we have at least one modifier in pro_mods
    if compounds['pro_mods'].isna().iloc[idx] == False:
        try:
            pro_mods =  " und ".join([el[0] for el in compounds['pro_mods'].iloc[idx].most_common(2) if el[1] > 1])
                #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
            if pro_mods != "":
                text += str_mods_pro
        
        except:
            pro_mods = "".join([el[0] for el in compounds['pro_mods'].iloc[idx].most_common(1) if el[1] > 1])
            if pro_mods != "":
            #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
                text += str_mods_pro
    else:
        print("NO PRO MOD FOUND")
        pro_mods = ""   
            
    # if we have at least one modifier in pro_mods
    if compounds['con_mods'].isna().iloc[idx] == False:
        try:
            con_mods =  " und ".join([el[0] for el in compounds['con_mods'].iloc[idx].most_common(2) if el[1] > 1])
            if con_mods != "":
                #print(con_mods)
                #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
                text += str_mods_con
        
        except:
            #pass
            con_mods = "".join([el[0] for el in compounds['con_mods'].iloc[idx].most_common(1) if el[1] > 1])
            if con_mods != "":
                
            #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
                text += str_mods_con
    else:
        print("NO CON MOD FOUND")
        con_mods = ""     
            
        
    # retrieve number of persons. if 0 then do nothing
        
    # if we have at least one person in PERS_con
    if compounds['PERS_con'].isna().iloc[idx] == False:
        try:
            con_pers =  " und ".join([el[0] for el in compounds['PERS_con'].iloc[idx].most_common(2)])
                #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
            text += str_pers_con
        except:
            con_pers = compounds['PERS_con'].iloc[idx].most_common(1)[0]
            #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
            text += str_pers_con
    else:
        print("NO PERSON FOUND")
            #con_pers = " " 
            
        # if we have at least one person in PERS_con
    if compounds['PERS_pro'].isna().iloc[idx] == False:
        try:
                #print("2 PERSONS FOUND")
            pro_pers = " und ".join([el[0] for el in compounds['PERS_pro'].iloc[idx].most_common(2)])
            #text +=  """und {PRO_PERS} (Pro Corpus). """
            text += str_pers_pro


        except:
            print("ONLY 1 PERSON FOUND")
            pro_pers = compounds['PERS_pro'].iloc[idx].most_common(1)[0]
            #text +=  """und {PRO_PERS} (Pro Corpus). """
            text += str_pers_pro
                
    else:
        pro_pers = " "
        print("NO PERSON FOUND")
            
    if compounds['ORG_con'].isna().iloc[idx] == False:
        try:
            con_org =  ", ".join([el[0] for el in compounds['ORG_con'].iloc[idx].most_common(2)])
            #text +=  """\nAußerdem werden im Kontext von {COMPOUND} folgende Organisationen genannt: {CON_ORG} (C2022)"""
            text += str_org_con
        except:
            con_org = compounds['ORG_con'].iloc[idx].most_common(1)[0]
            #text +=  """\nAußerdem werden im Kontext von {COMPOUND} folgende Organisationen genannt: {CON_ORG} (C2022)"""
            text += str_org_con 
                
    else:
        con_org = " " 
            
    # if we have at least one person in PERS_con
    if compounds['ORG_pro'].isna().iloc[idx] == False:
        try:
            pro_org =  ", ".join([el[0] for el in compounds['ORG_pro'].iloc[idx].most_common(2)])
            #text +=  """und {PRO_ORG} (P2022). """
            text += str_org_pro
        except:
            pro_org = compounds['ORG_pro'].iloc[idx].most_common(1)[0]
            #text +=  """und {PRO_ORG} (P2022). """   
            text += str_org_pro
                
    else:
        pro_org = " "
        #text += "."
            
    if len(compounds['similar_words'].iloc[idx]) != 0:
        #print(len(compounds['similar_words'].iloc[idx]))
        similar_words = set(["Klima"+x for x in compounds['similar_words'].iloc[idx] if "Klima"+x != compound])
        similar_words = ", ".join(similar_words)
        # similar_words = ", ".join(compounds['similar_words'].iloc[idx])
        #text += """\nSiehe auch: {SIMILAR_WORDS}"""
        text += str_simwords
            
            
    else:
        print("NO SIMILAR WORDS")
        similar_words = ""
            
    full_definition = text.format(COMPOUND= compound, DEFINITION= definition, CON_FREQ= con_freq,
                        PRO_FREQ = pro_freq, SENTIMENT= sentiment, CON_PERS = con_pers, PRO_PERS = pro_pers, 
                        CON_ORG = con_org, PRO_ORG = pro_org, SIMILAR_WORDS= similar_words, PRO_MODS = pro_mods, CON_MODS = con_mods)
        
        
    # full_definition = persons_text.format(COMPOUND= compound, DEFINITION= definition, CON_FREQ= con_freq,
    #                 PRO_FREQ = pro_freq, SENTIMENT= sentiment, CON_PERS = con_pers, PRO_PERS = pro_pers, 
        #                CON_ORG = con_org, PRO_ORG = pro_org, SIMILAR_WORDS= similar_words)
        
    print(full_definition)
    print("_"*50)
                
    # save to column "full_definition"
        
        
        

NO PRO MOD FOUND
NO PERSON FOUND
NO SIMILAR WORDS
Klimaabzockerei bezeichnet eine/n der Akt des Stehlens [Aktion] im Bezug auf den Klimawandel.
Der Begriff wird in unserem Korpus 1 Mal von den Klimaskeptikern und 0 Mal von den Klimaaktivisten verwendet und ist negativ konnotiert. 
Personen, die im Zusammenhang mit dem Begriff erwähnt werden sind Werner Kirstein (C2022) 
__________________________________________________
NO DEFINITION AVAILABLE
Klimaaktivismus bezeichnet eine/n   [Aktion] im Bezug auf den Klimawandel.
Der Begriff wird in unserem Korpus 1 Mal von den Klimaskeptikern und 3 Mal von den Klimaaktivisten verwendet und ist positiv konnotiert. 
Personen, die im Zusammenhang mit dem Begriff erwähnt werden sind Greta Thunberg (C2022) und Ken Jebsen und Wolf Dieter Storl (P2022). und GLS Bank, BUNDjugend (P2022). 
Siehe auch: Klimaaktivist
__________________________________________________
Der Begriff Klimaaktivist bezeichnet eine/n ein militanter Reformer im Bezug auf den Klimawa

In [60]:
#def fill_persons(df):
    
for word in compounds.original:
    idx = compounds.index[compounds['original'] == word][0]
    
    if compounds['concept'].iloc[idx] == "person":
        text = str_base_pers        
        #text = """Der Begriff {COMPOUND} bezeichnet eine/n {DEFINITION} im Bezug auf den Klimawandel.\nDer Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet und ist {SENTIMENT} konnotiert. """
        
    elif compounds['concept'].iloc[idx] == "location":
        text = str_base_loc
    
    elif compounds['concept'].iloc[idx] == "group":
        text = str_base_group
        
    elif compounds['concept'].iloc[idx] == "abstraction":
        text = str_base_abstract
        
    elif compounds['concept'].iloc[idx] == "action":
        text = str_base_action
    
        
    # retrieve according information
    compound = word.capitalize()
    con_freq = compounds['tf_con'].iloc[idx]
    pro_freq = compounds['tf_pro'].iloc[idx]
    sentiment = compounds['manual_sentiment'].iloc[idx]
        
    try:
        definition = compounds['definition'].iloc[idx][0] # try [0] for definition
    except:
        print("NO DEFINITION AVAILABLE")
        definition = " "
            
        
    # retrieve number of persons. if 0 then do nothing
        
    # if we have at least one person in PERS_con
    if compounds['PERS_con'].isna().iloc[idx] == False:
        try:
            con_pers =  " und ".join([el[0] for el in compounds['PERS_con'].iloc[idx].most_common(2)])
                #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
            text += str_pers_con
        except:
            con_pers = compounds['PERS_con'].iloc[idx].most_common(1)[0]
            #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
            text += str_pers_con
    else:
        print("NO PERSON FOUND")
            #con_pers = " " 
            
        # if we have at least one person in PERS_con
    if compounds['PERS_pro'].isna().iloc[idx] == False:
        try:
                #print("2 PERSONS FOUND")
            pro_pers = " und ".join([el[0] for el in compounds['PERS_pro'].iloc[idx].most_common(2)])
            #text +=  """und {PRO_PERS} (Pro Corpus). """
            text += str_pers_pro


        except:
            print("ONLY 1 PERSON FOUND")
            pro_pers = compounds['PERS_pro'].iloc[idx].most_common(1)[0]
            #text +=  """und {PRO_PERS} (Pro Corpus). """
            text += str_pers_pro
                
    else:
        pro_pers = " "
        print("NO PERSON FOUND")
            
    if compounds['ORG_con'].isna().iloc[idx] == False:
        try:
            con_org =  ", ".join([el[0] for el in compounds['ORG_con'].iloc[idx].most_common(2)])
            #text +=  """\nAußerdem werden im Kontext von {COMPOUND} folgende Organisationen genannt: {CON_ORG} (C2022)"""
            text += str_org_con
        except:
            con_org = compounds['ORG_con'].iloc[idx].most_common(1)[0]
            #text +=  """\nAußerdem werden im Kontext von {COMPOUND} folgende Organisationen genannt: {CON_ORG} (C2022)"""
            text += str_org_con 
                
    else:
        con_org = " " 
            
    # if we have at least one person in PERS_con
    if compounds['ORG_pro'].isna().iloc[idx] == False:
        try:
            pro_org =  ", ".join([el[0] for el in compounds['ORG_pro'].iloc[idx].most_common(2)])
            #text +=  """und {PRO_ORG} (P2022). """
            text += str_org_pro
        except:
            pro_org = compounds['ORG_pro'].iloc[idx].most_common(1)[0]
            #text +=  """und {PRO_ORG} (P2022). """   
            text += str_org_pro
                
    else:
        pro_org = " "
        #text += "."
            
    if len(compounds['similar_words'].iloc[idx]) != 0:
        #print(len(compounds['similar_words'].iloc[idx]))
        similar_words = set(["Klima"+x for x in compounds['similar_words'].iloc[idx] if "Klima"+x != compound])
        similar_words = ", ".join(similar_words)
        # similar_words = ", ".join(compounds['similar_words'].iloc[idx])
        #text += """\nSiehe auch: {SIMILAR_WORDS}"""
        text += str_simwords
            
            
    else:
        print("NO SIMILAR WORDS")
        similar_words = ""
            
    full_definition = text.format(COMPOUND= compound, DEFINITION= definition, CON_FREQ= con_freq,
                        PRO_FREQ = pro_freq, SENTIMENT= sentiment, CON_PERS = con_pers, PRO_PERS = pro_pers, 
                        CON_ORG = con_org, PRO_ORG = pro_org, SIMILAR_WORDS= similar_words)
        
        
    # full_definition = persons_text.format(COMPOUND= compound, DEFINITION= definition, CON_FREQ= con_freq,
    #                 PRO_FREQ = pro_freq, SENTIMENT= sentiment, CON_PERS = con_pers, PRO_PERS = pro_pers, 
        #                CON_ORG = con_org, PRO_ORG = pro_org, SIMILAR_WORDS= similar_words)
        
    print(full_definition)
    print("_"*50)
                
    # save to column "full_definition"
        
        
        

NO PERSON FOUND
NO SIMILAR WORDS
Klimaabzockerei bezeichnet eine/n der Akt des Stehlens im Bezug auf den Klimawandel.
Der Begriff wird in unserem Korpus 1 Mal von den Klimaskeptikern und 0 Mal von den Klimaaktivisten verwendet und ist negativ konnotiert. 
Personen, die im Zusammenhang mit dem Begriff erwähnt werden sind Werner Kirstein (C2022) 
__________________________________________________
NO DEFINITION AVAILABLE
Klimaaktivismus bezeichnet eine/n   im Bezug auf den Klimawandel.
Der Begriff wird in unserem Korpus 1 Mal von den Klimaskeptikern und 3 Mal von den Klimaaktivisten verwendet und ist positiv konnotiert. 
Personen, die im Zusammenhang mit dem Begriff erwähnt werden sind Greta Thunberg (C2022) und Ken Jebsen und Wolf Dieter Storl (P2022). und GLS Bank, BUNDjugend (P2022). 
Siehe auch: Klimaaktivist
__________________________________________________
Der Begriff Klimaaktivist bezeichnet eine/n ein militanter Reformer im Bezug auf den Klimawandel.
Der Begriff wird in unserem K

# TO DELETE?

In [155]:
#print(persons_text.format(COMPOUND="Klimaleugner", DEFINITION="XXX", CON_FREQ="123",
 #                        PRO_FREQ ="456", SENTIMENT="negative", 
  #                       CON_PERS = "Greta", PRO_PERS="Rahmstorf",
   #                      CON_ORG = "EIKE", PRO_ORG = "FFF",
    #                     SIMILAR_WORDS="Klimaleugnung"))

In [94]:
#compounds[compounds.original == compound].concept[0]

#idx = compounds.index[compounds['original'] == "klimaleugner"][0]
#compounds['concept'].iloc[idx]

In [184]:
#def fill_persons(df):
    
for word in compounds.original[:5]:
    #print(compound)
    idx = compounds.index[compounds['original'] == word][0]
    
    if compounds['concept'].iloc[idx] == "person":
        #print(compound)
        
        
        # retrieve according information
        compound = word.capitalize()
        con_freq = compounds['tf_con'].iloc[idx]
        pro_freq = compounds['tf_pro'].iloc[idx]
        sentiment = compounds['manual_sentiment'].iloc[idx]
        similar_words = compounds['similar_words'].iloc[idx]
        
        try:
            definition = compounds['definition'].iloc[idx][0] # try [0] for definition
        except:
            definition = " "
            
        
        # retrieve number of persons. if 0 then do nothing
        
        # if we have at least one person in PERS_con
        if len(compounds['PERS_con'].iloc[idx]) > 0:
            try:
                con_pers = compounds['PERS_con'].iloc[idx].most_common(2)
            except:
                con_pers = compounds['PERS_con'].iloc[idx].most_common(1)[0]
                
        else:
            con_pers = " " 
            
        # if we have at least one person in PERS_con
        if len(compounds['PERS_pro'].iloc[idx]) > 0:
            try:
                pro_pers = compounds['PERS_pro'].iloc[idx].most_common(2)
            except:
                pro_pers = compounds['PERS_pro'].iloc[idx].most_common(1)[0]
                
        else:
            con_pers = " " 
            
        if len(compounds['ORG_con'].iloc[idx]) > 0:
            try:
                con_org = compounds['ORG_con'].iloc[idx].most_common(2)
            except:
                con_org = compounds['ORG_con'].iloc[idx].most_common(1)[0]
                
        else:
            con_org = " " 
            
        # if we have at least one person in PERS_con
        if len(compounds['ORG_pro'].iloc[idx]) > 0:
            try:
                pro_org = compounds['ORG_pro'].iloc[idx].most_common(2)
            except:
                pro_org = compounds['ORG_pro'].iloc[idx].most_common(1)[0]
                
        else:
            pro_org = " "  
            
            
        
            
        
        con_pers1 = [el[0] for el in compounds['PERS_con'].iloc[idx].most_common(2)][0]
        con_pers2 = [el[0] for el in compounds['PERS_con'].iloc[idx].most_common(2)][1]
        pro_pers1 = [el[0] for el in compounds['PERS_pro'].iloc[idx].most_common(2)][0]
        pro_pers2 = [el[0] for el in compounds['PERS_pro'].iloc[idx].most_common(2)][1]

        con_org1 = [el[0] for el in compounds['ORG_con'].iloc[idx].most_common(2)][0]
        con_org2 = [el[0] for el in compounds['ORG_con'].iloc[idx].most_common(2)][1]
        pro_org1 = [el[0] for el in compounds['ORG_pro'].iloc[idx].most_common(2)][0]
        pro_org2 = [el[0] for el in compounds['ORG_pro'].iloc[idx].most_common(2)][1]
            
        con_freq = compounds['tf_con'].iloc[idx]
        pro_freq = compounds['tf_pro'].iloc[idx]
        sentiment = compounds['manual_sentiment'].iloc[idx]
        
        
        similar_words = compounds['similar_words'].iloc[idx]
        
        full_definition = persons_text.format(COMPOUND= compound, DEFINITION= definition, CON_FREQ= con_freq,
                         PRO_FREQ = pro_freq, SENTIMENT= sentiment, CON_PERS1 = con_pers1, CON_PERS2 = con_pers2,
                         PRO_PERS1 = pro_pers1, PRO_PERS2 = pro_pers2, CON_ORG1 = con_org1, CON_ORG2 = con_org2, 
                         PRO_ORG1 = pro_org1, PRO_ORG2 = pro_org2, SIMILAR_WORDS= similar_words)
        
        print(full_definition)
        print("_"*50)
                
        # save to column "full_definition"
        
        
        

Der Begriff Klimaaktivist bezeichnet eine/n ein militanter Reformer im Bezug auf den Klimawandel. 
Der Begriff wird in unserem Korpus 66 Mal von den Klimaskeptikern und 61 Mal von den Klimaaktivisten verwendet
 und ist positive konnotiert. Personen, die im Zusammenhang mit dem Begriff erwähnt werden sind 
 Greta Thunberg und Bill McKibben (Con Corpus) und Greta Thunberg und Vanessa Nakate (Pro Corpus). 
 Außerdem werden im Kontext von Klimaaktivist folgende Organisationen genannt: EIKE und IPCC (C2022), sowie FFF und EU (P2022).

Siehe auch: ['demagoge', 'macher', 'aktivist', 'aktivismus']
__________________________________________________
Der Begriff Klimaaktivistin bezeichnet eine/n   im Bezug auf den Klimawandel. 
Der Begriff wird in unserem Korpus 9 Mal von den Klimaskeptikern und 24 Mal von den Klimaaktivisten verwendet
 und ist positive konnotiert. Personen, die im Zusammenhang mit dem Begriff erwähnt werden sind 
 Greta Thunberg und Naomi Seibt (Con Corpus) und Greta Thunberg un

In [188]:
persons_text = """Der Begriff {COMPOUND} bezeichnet eine/n {DEFINITION} im Bezug auf den Klimawandel. 
Der Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet
 und ist {SENTIMENT} konnotiert. Personen, die im Zusammenhang mit dem Begriff erwähnt werden sind 
 {CON_PERS} (Con Corpus) und {PRO_PERS} (Pro Corpus). 
 Außerdem werden im Kontext von {COMPOUND} folgende Organisationen genannt: {CON_ORG} (C2022), sowie {PRO_ORG} (P2022).

Siehe auch: {SIMILAR_WORDS}"""

In [222]:
idx = compounds.index[compounds['original'] == "klimaanbeter"][0]
compounds['PERS_con'].iloc[idx] == pd.isna



compounds['PERS_con'].isna().iloc[idx]

True

In [379]:
for word in compounds.original:
    #print(compound)
    idx = compounds.index[compounds['original'] == word][0]

    if compounds['PERS_con'].isna().iloc[idx] != False:
        #print(compounds['PERS_con'].iloc[idx])
        pass

In [ ]:
i = 0
for n in compounds.definition:
    if len(n) == 0:
        i += 1
        
print(i)

41


### IDEA:
Initiate string pieces that can be put together depending on the situation and available information

-> work on definition part here? or retrieve definitions for all words

#def fill_persons(df):
    
for word in compounds.original:
    #print(compound)
    idx = compounds.index[compounds['original'] == word][0]
    
    if compounds['concept'].iloc[idx] == "person":
        #print(compound)
        
        #text = """Der Begriff {COMPOUND} bezeichnet eine/n {DEFINITION} im Bezug auf den Klimawandel.\nDer Begriff wird in unserem Korpus {CON_FREQ} Mal von den Klimaskeptikern und {PRO_FREQ} Mal von den Klimaaktivisten verwendet und ist {SENTIMENT} konnotiert. """
        text = str_base_pers
        
        # retrieve according information
        compound = word.capitalize()
        con_freq = compounds['tf_con'].iloc[idx]
        pro_freq = compounds['tf_pro'].iloc[idx]
        sentiment = compounds['manual_sentiment'].iloc[idx]
        
        try:
            definition = compounds['definition'].iloc[idx][0] # try [0] for definition
        except:
            #print("NO DEFINITION AVAILABLE")
            definition = " "
            
        
        # retrieve number of persons. if 0 then do nothing
        
        # if we have at least one person in PERS_con
        if compounds['PERS_con'].isna().iloc[idx] == False:
            try:
                con_pers =  " und ".join([el[0] for el in compounds['PERS_con'].iloc[idx].most_common(2)])
                #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
                text += str_pers_con
            except:
                con_pers = compounds['PERS_con'].iloc[idx].most_common(1)[0]
                #text += """\nPersonen, die im Zusammenhang mit dem Begriff erwähnt werden sind {CON_PERS} (Con Corpus) """
                text += str_pers_con
        else:
          #  print("NO PERSON FOUND")
            #con_pers = " " 
            
        # if we have at least one person in PERS_con
        if compounds['PERS_pro'].isna().iloc[idx] == False:
            try:
                #print("2 PERSONS FOUND")
                pro_pers = " und ".join([el[0] for el in compounds['PERS_pro'].iloc[idx].most_common(2)])
                #text +=  """und {PRO_PERS} (Pro Corpus). """
                text += str_pers_pro


            except:
                print("ONLY 1 PERSON FOUND")
                pro_pers = compounds['PERS_pro'].iloc[idx].most_common(1)[0]
                #text +=  """und {PRO_PERS} (Pro Corpus). """
                text += str_pers_pro
                
        else:
            pro_pers = " "
            #print("NO PERSON FOUND")
            
        if compounds['ORG_con'].isna().iloc[idx] == False:
            try:
                con_org =  ", ".join([el[0] for el in compounds['ORG_con'].iloc[idx].most_common(2)])
                #text +=  """\nAußerdem werden im Kontext von {COMPOUND} folgende Organisationen genannt: {CON_ORG} (C2022)"""
                text += str_org_con
            except:
                con_org = compounds['ORG_con'].iloc[idx].most_common(1)[0]
                #text +=  """\nAußerdem werden im Kontext von {COMPOUND} folgende Organisationen genannt: {CON_ORG} (C2022)"""
                text += str_org_con 
                
        else:
            con_org = " " 
            
        # if we have at least one person in PERS_con
        if compounds['ORG_pro'].isna().iloc[idx] == False:
            try:
                pro_org =  ", ".join([el[0] for el in compounds['ORG_pro'].iloc[idx].most_common(2)])
                #text +=  """und {PRO_ORG} (P2022). """
                text += str_org_pro
            except:
                pro_org = compounds['ORG_pro'].iloc[idx].most_common(1)[0]
                #text +=  """und {PRO_ORG} (P2022). """   
                text += str_org_pro
                
        else:
            pro_org = " "
            #text += "."
            
        if len(compounds['similar_words'].iloc[idx]) != 0:
            #print(len(compounds['similar_words'].iloc[idx]))
            similar_words = set(["Klima"+x for x in compounds['similar_words'].iloc[idx] if "Klima"+x != compound])
            similar_words = ", ".join(similar_words)
           # similar_words = ", ".join(compounds['similar_words'].iloc[idx])
            #text += """\nSiehe auch: {SIMILAR_WORDS}"""
            text += str_simwords
            
            
        else:
           # print("NO SIMILAR WORDS")
            
        full_definition = text.format(COMPOUND= compound, DEFINITION= definition, CON_FREQ= con_freq,
                        PRO_FREQ = pro_freq, SENTIMENT= sentiment, CON_PERS = con_pers, PRO_PERS = pro_pers, 
                        CON_ORG = con_org, PRO_ORG = pro_org, SIMILAR_WORDS= similar_words)
        
        
       # full_definition = persons_text.format(COMPOUND= compound, DEFINITION= definition, CON_FREQ= con_freq,
        #                 PRO_FREQ = pro_freq, SENTIMENT= sentiment, CON_PERS = con_pers, PRO_PERS = pro_pers, 
         #                CON_ORG = con_org, PRO_ORG = pro_org, SIMILAR_WORDS= similar_words)
        
        #print(full_definition)
        #print("_"*50)
                
        # save to column "full_definition"
        
        
        

# TO DELETE: